# Exercice 3 — Orchestration du pipeline avec dbt (DuckDB)

Ce notebook met en place une architecture **médaillon** (bronze / silver / gold) et exécute dbt localement sur une base **DuckDB**.
Les données sources sont lues depuis des **Parquet** produits à l’exercice 1.

Pré-requis :
- un environnement Python 3.13 (ex: `.venv313`)
- `dbt-core`, `dbt-duckdb`, `duckdb`

##  Paramètres et chemins

In [1]:
from pathlib import Path

RACINE_EX3 = Path(r"C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3")
RACINE_EX1 = Path(r"C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1")

# Projet dbt
DBT_DIR = RACINE_EX3 / "dbt_trafic"
PROFILES_DIR_LOCAL = RACINE_EX3 / ".dbt"

# DuckDB (datalake local)
WAREHOUSE_DB = RACINE_EX3 / "data" / "warehouse.duckdb"

# Exécutable dbt dans le venv 3.13
DBT_EXE = RACINE_EX3 / ".venv313" / "Scripts" / "dbt.exe"

PARQUET_TRAFIC_GLOB = RACINE_EX1 / "data" / "parquet" / "ci_trafi_l" / "*.parquet"
PARQUET_EVENTS_GLOB = RACINE_EX1 / "data" / "parquet" / "ci_evenmt_p" / "*.parquet"
PARQUET_VAC_GLOB    = RACINE_EX1 / "data" / "parquet" / "calendrier_scolaire" / "*.parquet"
PARQUET_FERIES_GLOB = RACINE_EX1 / "data" / "parquet" / "jours_feries_metropole" / "*.parquet"

print("DBT_DIR =", DBT_DIR)
print("PROFILES_DIR_LOCAL =", PROFILES_DIR_LOCAL)
print("WAREHOUSE_DB =", WAREHOUSE_DB)
print("DBT_EXE =", DBT_EXE)


DBT_DIR = C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\dbt_trafic
PROFILES_DIR_LOCAL = C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.dbt
WAREHOUSE_DB = C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\data\warehouse.duckdb
DBT_EXE = C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.venv313\Scripts\dbt.exe


## 1) Créer l’arborescence du projet dbt

In [2]:
# Dossiers dbt
(DBT_DIR / "models" / "bronze").mkdir(parents=True, exist_ok=True)
(DBT_DIR / "models" / "silver").mkdir(parents=True, exist_ok=True)
(DBT_DIR / "models" / "gold").mkdir(parents=True, exist_ok=True)

# Dossiers profils + warehouse
PROFILES_DIR_LOCAL.mkdir(parents=True, exist_ok=True)
WAREHOUSE_DB.parent.mkdir(parents=True, exist_ok=True)

print("OK - dossiers créés")


OK - dossiers créés


## 2) Générer `profiles.yml` (DuckDB)

In [3]:
profiles_yml = f"""
trafic_bordeaux:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: "{WAREHOUSE_DB.as_posix()}"
      schema: main
      threads: 4
""".lstrip()

(PROFILES_DIR_LOCAL / "profiles.yml").write_text(profiles_yml, encoding="utf-8")
print("OK - profiles.yml écrit :", (PROFILES_DIR_LOCAL / "profiles.yml"))


OK - profiles.yml écrit : C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.dbt\profiles.yml


## 3) Générer `dbt_project.yml`


In [4]:
dbt_project_yml = """
name: "trafic_bordeaux"
version: "1.0.0"
config-version: 2

profile: "trafic_bordeaux"

model-paths: ["models"]

models:
  trafic_bordeaux:
    bronze:
      +materialized: view
    silver:
      +materialized: table
    gold:
      +materialized: table
""".lstrip()

(DBT_DIR / "dbt_project.yml").write_text(dbt_project_yml, encoding="utf-8")
print("OK - dbt_project.yml écrit :", (DBT_DIR / "dbt_project.yml"))


OK - dbt_project.yml écrit : C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\dbt_trafic\dbt_project.yml


## 4) Créer les modèles `bronze` (lecture directe des Parquet)


In [5]:
BRONZE_DIR = DBT_DIR / "models" / "bronze"

def write_bronze(model_name: str, parquet_glob: Path):
    parquet_glob_posix = parquet_glob.as_posix()
    sql = f"select * from read_parquet('{parquet_glob_posix}')\n"
    (BRONZE_DIR / f"{model_name}.sql").write_text(sql, encoding="utf-8")
    print("OK -", model_name, "->", parquet_glob_posix)

write_bronze("bronze_ci_trafi_l", PARQUET_TRAFIC_GLOB)
write_bronze("bronze_ci_evenmt_p", PARQUET_EVENTS_GLOB)
write_bronze("bronze_calendrier_scolaire", PARQUET_VAC_GLOB)
write_bronze("bronze_jours_feries", PARQUET_FERIES_GLOB)


OK - bronze_ci_trafi_l -> C:/Users/HP-X360-1030-G3/PycharmProjects/notebooks/exercice1/data/parquet/ci_trafi_l/*.parquet
OK - bronze_ci_evenmt_p -> C:/Users/HP-X360-1030-G3/PycharmProjects/notebooks/exercice1/data/parquet/ci_evenmt_p/*.parquet
OK - bronze_calendrier_scolaire -> C:/Users/HP-X360-1030-G3/PycharmProjects/notebooks/exercice1/data/parquet/calendrier_scolaire/*.parquet
OK - bronze_jours_feries -> C:/Users/HP-X360-1030-G3/PycharmProjects/notebooks/exercice1/data/parquet/jours_feries_metropole/*.parquet


## 5) Créer les modèles `silver` (stubs propres)


In [6]:
SILVER_DIR = DBT_DIR / "models" / "silver"

silver_trafic = """
select *
from {{ ref('bronze_ci_trafi_l') }}
""".strip() + "\n"

silver_calendrier = """
select *
from {{ ref('bronze_jours_feries') }}
""".strip() + "\n"

(SILVER_DIR / "silver_trafic.sql").write_text(silver_trafic, encoding="utf-8")
(SILVER_DIR / "silver_calendrier.sql").write_text(silver_calendrier, encoding="utf-8")

print("OK - modèles silver écrits")


OK - modèles silver écrits


## 6) Créer le modèle `gold` (stub)

In [7]:
GOLD_DIR = DBT_DIR / "models" / "gold"

gold_training = """
select *
from {{ ref('silver_trafic') }}
""".strip() + "\n"

(GOLD_DIR / "gold_training_set.sql").write_text(gold_training, encoding="utf-8")
print("OK - modèle gold écrit")


OK - modèle gold écrit


## 7) Exécuter dbt 


In [8]:
import os
import subprocess

def run_dbt(args: list[str]) -> int:
    env = os.environ.copy()
    env["DBT_PROFILES_DIR"] = str(PROFILES_DIR_LOCAL)

    cmd = [str(DBT_EXE)] + args
    print(">", " ".join(cmd))

    r = subprocess.run(cmd, cwd=str(DBT_DIR), env=env, text=True, capture_output=True)
    print(r.stdout)
    if r.returncode != 0:
        print(r.stderr)
    return r.returncode

run_dbt(["debug"])
run_dbt(["run"])


> C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.venv313\Scripts\dbt.exe debug
14:22:53  Running with dbt=1.10.16
14:22:53  dbt version: 1.10.16
14:22:53  python version: 3.13.7
14:22:53  python path: C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.venv313\Scripts\python.exe
14:22:53  os info: Windows-11-10.0.26200-SP0
14:22:53  Using profiles dir at C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.dbt
14:22:53  Using profiles.yml file at C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\.dbt\profiles.yml
14:22:53  Using dbt_project.yml file at C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice3\dbt_trafic\dbt_project.yml
14:22:53  adapter type: duckdb
14:22:53  adapter version: 1.10.0
14:22:53  Configuration:
14:22:53    profiles.yml file [OK found and valid]
14:22:53    dbt_project.yml file [OK found and valid]
14:22:53  Required dependencies:
14:22:53   - git [OK found]

14:22:53  Connection:
14:22:53    database: warehou

0

## 8) Vérification rapide dans DuckDB


In [9]:
import sys
!{sys.executable} -m pip install -U numpy pandas


In [10]:
import duckdb

con = duckdb.connect(str(WAREHOUSE_DB))

print(con.execute("show tables").fetchdf())
print(con.execute("select count(*) as nb_lignes from gold_training_set").fetchdf())


                         name
0  bronze_calendrier_scolaire
1          bronze_ci_evenmt_p
2           bronze_ci_trafi_l
3         bronze_jours_feries
4           gold_training_set
5           silver_calendrier
6               silver_trafic
   nb_lignes
0       1388
